<a href="https://colab.research.google.com/github/math77/ml-classification-basic/blob/main/ml_intro_casadocodigo_book.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Cap. 01

In [ ]:
"""
100% success rate practically impossible to achieve in real life. This algorithm
is very easy to be wrong.
"""

from sklearn.naive_bayes import MultinomialNB

pig1 = [1, 1, 0]
pig2 = [1, 1, 0]
pig3 = [1, 1, 0]
dog1 = [1, 1, 1]
dog2 = [0, 1, 1]
dog3 = [0, 1, 1]

data = [pig1, pig2, pig3, dog1, dog2, dog3]

marks = [1, 1, 1, -1, -1, -1]

test = [[1, 1, 1], [1, 0, 0], [0, 0, 1]]
marks_test = [-1, 1, -1]

model = MultinomialNB()

# Adapt the model to our data and markings
model.fit(data, marks)
result = model.predict(test)

difference = result - marks_test

matching = [d for d in difference if d == 0]

matching_rate = 100.0 * len(matching) / len(test)

print(result)
print(matching_rate)


[-1  1 -1]
100.0


# Cap. 02


In [ ]:
import csv

from sklearn.naive_bayes import MultinomialNB

def calc_percent(result, X, Y):
  difference = result - Y
  matching = [d for d in difference if d == 0]
  
  return 100.0 * len(matching) / len(X)


def data_load():
  X = []
  Y = []

  file_read = open('/content/acesso.csv', 'r')
  reader = csv.reader(file_read)

  # Ignore the header by skipping to the next line.
  next(reader)

  for acessou_home, acessou_como_funciona, acessou_contato, comprou in reader:
    X.append([int(acessou_home), int(acessou_como_funciona), int(acessou_contato)])
    Y.append(int(comprou))
  
  return X, Y


X, Y = data_load()
X_train = X[:90]
Y_train = Y[:90]

X_test = X[-9:]
Y_test = Y[-9:]

model = MultinomialNB()
model.fit(X_train, Y_train)

result = model.predict(X_test)
print(calc_percent(result, X_test, Y_test))
print(len(X_test))

88.88888888888889
9


# Cap. 03

In [11]:
from sklearn.naive_bayes import MultinomialNB
import pandas as pd


def calc_percent(result, X, Y):
  difference = result - Y
  matching = [d for d in difference if d == 0]

  return 100.0 * len(matching) / len(X)


#df -> dataframe
df = pd.read_csv('/content/buscas.csv')
X_df = df[['home', 'busca', 'logado']]
Y_df = df['comprou']
Xdummies_df = pd.get_dummies(X_df)
Ydummies_df = Y_df

X = Xdummies_df.values
Y = Ydummies_df.values

train_size = int(0.9 * len(Y))
test_size = len(Y) - train_size

train_data = X[:train_size]
train_marks = Y[:train_size]

test_data = X[-test_size:]
test_marks = Y[-test_size:]

model = MultinomialNB()
model.fit(train_data, train_marks)

result = model.predict(test_data)

print(calc_percent(result, test_data, test_marks))

82.0
